Version 0.0.2.9.2

In [ ]:
import sys
sys.path.append("..")

from include.RandomHelper import check_data_state
check_data_state()

In [ ]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {return false;}

Im ersten Aufgabenteil sollten die zur Verfügung gestellten Datensätze betrachtet und derart reduziert werden, dass nur noch ein bestimmter Phasenraum übrigbleibt - der Ereignisse enthält, die eine signalähnliche Signatur aufweisen - ein Zerfall in vier Leptonen über ein Z Bosonenpaar, wobei ein Z-Boson virtuell ist.

Als letzten Schritt wurde eine sehr grobe Abschätzung der Signifikanz durchgeführt ($Z \approx s/\sqrt{b}$) und diskutiert, warum diese ungeeignet ist.

In diesem Aufgabenteil werden zwei Methoden vorgestellt, wie die Signifikanz einer Messung bestimmt werden kann. Die erste Methode (lokaler $p_0$-Wert) wurde hierbei bei der Entdeckung des Higgs-Bosons verwendet - in diesem Aufgabenteil wird eine vereinfachte Version hiervon vorgestellt. Die zweite Methode bestimmt die Signifikanz mithilfe der zu bestimmenden Signalwahrscheinlichkeit.

Bei beiden Methoden werden wir die Frage nach der systematischen Unsicherheit nicht betrachtet, da durch die Hinzunahme höherdimensionale Anpassungen durchgeführt werden müssen, die nicht unbedingt komplizierter, jedoch viel Zeitintensiver sind.

Sie können die erhaltenen Datensätze nun hier weiterverwenden.

# Parametrisierung des Untergrundes
------------------------------------------------------------

Für die Vermeidung von Fluktuationen ist es vorteilhaft die Untergrundverteilung der MC-Simulationen im ersten Schritt durch eine Wahrscheinlichkeitsdichte auszudrücken.

Hierzu soll nochmal der Unterschied zwischen einer Anpassung und einer Parametrisierung verdeutlicht werden: Die Anwendung einer Anpassung geschieht immer dann, wenn eine bestimmte, vorher dazu aufgestellte Theorie durch die gemessenen Daten bestätigt oder falsifiziert werden soll. Die für die Anpassung enthaltenen Daten sind mit einer Unsicherheit behaftet, die vom Experiment und Messmethoden abhängig, aber vor allem durch die Messung der Daten fest vorgegeben ist.

Bei einer Parametrisierung ist das Ziel die vorhandene Messung, die in diesem Beispiel in Form eines Histogramms zur Verfügung steht, durch eine andere Form auszudrücken. In diesem Fall soll es eine kontinuierliche Funktion sein - genauer: eine kontinuierliche Wahrscheinlichkeitsdichte in einem bestimmten Intervall.

Damit ist der Hauptunterschied zwischen einer Anpassung und einer Parametrisierung deutlich: Während im ersten Fall eine von der Theorie vorgegebene Abhängigkeit vorliegt wird im zweiten Fall versucht eine Parametrisierung zu finden, die möglichst genau die Daten in einer anderen Form beschreibt.

Es ist in diesem Sinne - und vor allem in diesem Fall - nicht möglich eine genaue Funktion hierfür zu finden. Um trotzdem eine Wahl treffen zu können, kann wie folgt vorgegangen werden: Bei einer $\chi^2$ - Anpassung sind die Unsicherheiten auf die Daten ausschlaggebend, ob ein passendes Modell zu den Daten gewählt werden kann ($\frac{\chi^2}{n_{\mathrm{df}}}$). Im Umkehrschluss können die Unsicherheiten auf die Daten skaliert werden, bis ein Modell passt. Die Frage, welches Modell die Daten besser parametrisiert, kann mit dem Skalierungsfaktor beantwortet werden.

Jedoch ist nicht nur der kleinste Skalierungsfaktor der ausschlaggebende Punkt, da dies immer durch ein Polynom eines sehr hohen Grades realisiert werden kann. Das sogenannte Overfitting soll dabei vermieden werden, da nur der Verlauf der Daten parametrisiert werden will und nicht noch zusätzlich deren Fluktuation, die in dieser Messung bzw. Simulation beobachtet wurde. Die Wahl sollte also auf eine Funktion fallen, die noch ausreichend wenige Parameter enthält, den Skalierungsfaktor für die Unsicherheit aber bereits möglichst klein hält.

<div class="alert alert-info">

Parametrisieren Sie den Untergrund durch ein geeignetes Modell. Stellen Sie zudem sicher, dass das gewählte Modell das passendste ist.
    
</div>

Die Implementierung von Linearkombinationen der Legendrepolynome ist bereits durchgeführt und in der Klasse `LLC` zusammengefasst. Ebenso können eigenständige Funktionen verwendet werden (siehe hierzu die numpy Dokumentation).

Die Benutzung der Linearkombination von Legendrepolynomen bieten im Gegensatz zu den normalen Polynomen den Vorteil, dass die Legendrepolynome ein orthogonales Funktionensystem bilden und sich robust bei einer Anpassung verhalten - anders als normale Polynome. Die Korrelation der Unsicherheiten auf die einzelnen Parameter wird aufgrund der Orthogonalität verringert, während bei normalen Polynomen fast immer eine starke Korrelation zwischen den einzelnen Parametern besteht.

In [ ]:
import matplotlib.pyplot as plt
from include.parameterization.LegendrePoly import LegendreLinearCombination as LLC

In [ ]:
llc_background = LLC(x_mean=0.0) # Zentrierung um x_mean

x_ = np.linspace(-1, 1, 1000)
plt.plot(x_, llc_background.grade_0(x_, a=1))
plt.plot(x_, llc_background.grade_1(x_, a=1, b=1))
plt.plot(x_, llc_background.grade_2(x_, a=1, b=1, c=1))

Vorbereitend sollen die Binanzahl und das betrachtete Intervall gewählt werden:

In [ ]:
bins, hist_range, info = 999, (999.1, 999.2), [["2012"], ["B", "C"]]
mc_dir = "../data/for_long_analysis/mc_aftH/"  
llc_background = LLC(x_mean=(999.1 + 999.2)/2.)

Variante:

 1. <ins>Die Daten aus dem Histogramm werden roh ausgegeben.</ins> 
<div class="alert alert-info">
     
    * Berechnen Sie die Unsicherheiten auf die einzelnen Bineinträge des skalierten Histogramms
    * Skalieren Sie die Histogramme der einzelnen Kanäle und fassen sie diese in einem gemeinsamen Histogramm zusammen.

    (Das Zugreifen auf die einzelnen Teile kann über den Attribut ´data´ geschehen, wobei das ein geschachteltes ´dict´ ist. Die erste Ebene enthält die drei einzelnen Kanäle. In jedem der Kanäle ist anschließend die unskalierten Histogrammeinträge, der Korrekturfaktor und die Daten, die für die Histogrammbildung verwendet wurden, gespeichert.)

</div>

Die so berechneten Unsicherheiten sind Grundverschieden von der $\sqrt{N}$ Variante - sie beschreiben die skalierte Fluktuation der Datenpunkte.

In [ ]:
from include.histogramm.HistDataGetter import HistDataGetter as HDG

In [ ]:
raw_mc_data = HDG(bins=bins, hist_range=hist_range, 
           info=info, mc_dir=mc_dir).get_mc_raw(tag="background")
print(list(raw_mc_data.data.keys()))
raw_mc_data.data["mc_track_ZZ_4el_2012"]

In [ ]:
import kafe2 as K2
# code

2. <ins>Die Daten aus dem Histogramm werden zusammengefasst ausgegeben</ins> 
    <div class="alert alert-info">
     
    Verwenden Sie die bereits zusammengefassten Größen für den Histogrammfit.

</div>

In [ ]:
from include.fits.McFitInit import McFitInit

In [ ]:
format_data = McFitInit(bins=bins, hist_range=hist_range, tag="background", mc_dir=mc_dir, info=info)
bins, hist_range, hist_entries, combined_hist_entries_errors = format_data.variables_for_hist_fit

Der eigentliche Fit erfolgt mithilfe des in 'kafe2' implementiertem Histogramfits. Ein Beispiel für einen Histogrammfit findet sich [hier](https://kafe2.readthedocs.io/en/latest/parts/user_guide.html#example-5-histogram-fit)

Es ist vorteilhaft sich eine Funktion oder Klasse zu erstellen, um umständliche Wiederholungen zu vermeiden.

In [ ]:
import kafe2 as K2
# code

# Auswahl der passenden  Signal MC
-----------------------------------------------------------

Das Ziel dieses Abschnittes ist eine Eingrenzung der möglichen Signal-MC-Simulationen durchzuführen und sich auf ein einziges festzulegen und damit weiterzuarbeiten.

## Vorläufige Parametrisierung der Signal MC
--------------------------------------------------------------

Es stehen unterschiedliche Signal-MC Simulationen zur Verfügung. Diese wurden durch eine Gaußverteilung $\mathcal{G} \left( m_{4\ell}, m_{\mathrm{H}}, \sigma_{\mathrm{G}} \right)$ grob parametrisiert. Das Ziel ist es die Abhängigkeit der Breite von der invarianten Masse festzustellen und durch ein passendes Modell zu beschreiben. Die Auflösung des Transversalimpulses des Detektors ist hierbei der größte Einfluss.

Die Ergebnisse dieser Parametrisierungen sind in einer csv Datei zusammengefasst:

In [ ]:
import pandas as pd
df_ = pd.read_csv("../data/for_long_analysis/other_mc_gauss_param/gauss_params.csv")
df_
sigma, sigma_err, mu, mu_err = tuple(df_[["sigma", "sigma_err", "mu", "mu_err"]].values.T)

<div class="alert alert-info">

Parametrisieren Sie $\sigma_{\mathrm{G}}(m_{4\ell})$ anhand der vorhandenen Daten.
  
</div>

In [ ]:
def sigma_model(x, *args):
    """
    Model to for sigma parameterization
    
    """
    # code
    
    return x

# code

Diese Variante - die kontinuierliche Parametrisierung der Signal MC findet in der richtigen Analyse nicht statt und wird hier vor allem aus zeitlichen Gründen verwendet. Normalerweise wird für jede Higgs Masse eine eigene MC Simulation erstellt, mit der alle weiteren Schritte durchgeführt werden. Jedoch ist das Erstellen derartiger Simulation äußerst zeitintensiv. Eine lineare Interpolation einer gaußartigen Parametrisierung führt aber bereits zu einem ausreichenden Ergebnis, den man im Rahmen dieses Abschnittes erreichen will: Die Begründung der Wahl einer bestimmten Signal MC Simulation anhand der Messung - ebenso wird eine Abschätzung der statistischen Signifikanz (dazu mehr in den nächsten Teilabschnitten).

## Likelihood - Quotienten - Test
----------------------------------------------

Für die Auswahl zwischen den unterschiedlichen Simulationen wird explizit der Likelihood-Quotienten-Test verwendet.
Hierzu werden die Nullhypothese $H_0$, die nur dem Untergrund entspricht, und die Gegenhypothese $H_1$, die dem Untergrund und einem Signal entspricht, miteinander verglichen.

In die extended Likelihood der Hypothese $H_0$ fließt die Parametrisierung des Untergrundes $f_{\mathrm{U}}(x|\theta_{\mathrm{U}})$ ein wobei $f_{\mathrm{U}}$ das vorher festgelegte Modell aus der Untergrundparametrisierung mit den optimalen Parametern $\theta_{\mathrm{U}}$ ist.

Die für die Gegenhypothese verwendete Verteilung lässt sich beschreiben durch: $$f_{\mu\mathrm{S+B}}(m_{4\ell}|\mu) = \mu f_{\mathrm{S}}(m_{4\ell}|\theta_{\mathrm{S}}) + f_{\mathrm{U}}(m_{4\ell}|\theta_{\mathrm{U}}) \, , \\ \int_{\Omega} f_{\mu\mathrm{S+B}}(m_{4\ell}|\mu) \mathrm{d}m_{4\ell} = 1 $$ $f_{\mathrm{S}}(x|\theta_{\mathrm{S}})$ ist für diesen Fall eine Gaußverteilung $\mathcal{G}(m_{4\ell}, m_{\mathrm{H}}, \sigma_{\mathrm{G}}(m_{\mathrm{H}} ))$ mit festgehaltenen Werten für den Schwerpunkt und die Breite. Für den Schätzer $\mu$ gilt: $$\mu \geq 0 \, .$$ Diese Einschränkung führt zu einer vernachlässigbaren Verzerrung und wird aus physikalischen Überlegungen eingeführt, da in der Verteilung der Invarianten Massen ein Überschuss an Ereignissen erwartet wird, wenn ein Higgs Boson vorhanden sein sollte.

Damit kann die zu betrachtende Größe – das Verhältnis aus den Likelihoods beider Hypothesen – gebildet werden. Für die numerische Stabilität wird der negative Logarithmus davon gebildet. Der Faktor zwei ist für die spätere asymptotische Form notwendig.

$$ q_0 = -2 \ln \left( \frac{\mathcal{L}(x|\mu = \hat{\mu})}{\mathcal{L}(x|\mu = 0)} \right) \, .$$ 
$\hat{\mu}$ ist hierbei der ideale Schätzer für eine feste Masse und Breite der Signalhypothese.

Die verwendete Teststatistik $q_0$ ist hierbei eine vereinfachte Version der am LHC für diese Analyse verwendeten Teststatistik. Die entspricht der am LEP verwendeten Teststatistik und hat den Nachteil der bereits erwähnten Verzerrung. Jedoch ist der Vorteil der Teststatistik, dass eine Implementierung dieser einfacher ist und die Berechnung weniger Zeit benötigt da keine Toy Monte Carlo Simulationen hierzu notwendig sind. Ebenso kann bereits für eine geringe Anzahl an Ereignissen die analytische Form verwendet werden, die die Teststatistik im Grenzfall vieler Ereignisse annimmt.

Zur Erinnerung: Die extended Likelihood Funktion ist wie folgt definiert: $$ \mathcal{L}(x|\mu) = \frac{\nu(\mu)^N}{N!}\mathrm{e}^{-\nu(\mu)}\prod_i^N f_{\mu\mathrm{S+B}}(x_i|\mu) $$ und unterscheidet sich in dem Skalierungsfaktor von der normalen Likelihood Funktion. Durch diesen Skalierungsfaktor spielt die Anzahl der Messungen (da es sich um ein Zählexperiment handelt) eine Rolle.

In [ ]:
lower_bound, upper_bound = hist_range
mass_array = np.linspace(lower_bound, upper_bound, 100)
sigma_model_arguments = tuple([])
sigma_array = sigma_model(mass_array, *sigma_model_arguments)

<div class="alert alert-info">

Implementieren Sie die den Quotienten Likelihood Test. Für die numerische Integration können Sie die Methode ['scipy.integrate.quad'](https://docs.scipy.org/doc/scipy/reference/generated/scipy.integrate.quad.html) verwenden. Die Minimierung der Likelihood Funktion können sie mithilfe von ['iminuit'](https://nbviewer.jupyter.org/github/scikit-hep/iminuit/blob/master/tutorial/basic_tutorial.ipynb
) durchführen.

</div>

In [ ]:
# Messung:
from include.histogramm.HistDataGetter import HistDataGetter as HDG

bins, hist_range, info = 9999, (999.1, 999.2), [["2012"], ["B", "C"]]
mc_dir = "../data/for_long_analysis/mc_aftH/" 
ru_dir = "../data/for_long_analysis/ru_aftH/"
raw_data = HDG(bins=bins, hist_range=hist_range, 
           info=info, mc_dir=mc_dir, ru_dir=ru_dir).get_data_raw("mass_4l")
data = raw_data.data
data

# Skalierung der MC-Simulationen an die Daten:
b_mc_d = McFitInit(bins=bins, hist_range=hist_range, tag="background", mc_dir=mc_dir)
_, _, background_hist, _ = b_mc_d.variables_for_hist_fit
scale_b_mc = np.sum(background_hist)

s_mc_d = McFitInit(bins=bins, hist_range=hist_range, tag="signal", mc_dir=mc_dir)
_, _, signal_hist, _ = s_mc_d.variables_for_hist_fit
scale_s_mc = np.sum(signal_hist)

scale_s_mc, scale_b_mc

In [ ]:
# Iminuit Example:
from iminuit import Minuit

def extended_likelihood(mu=0.0):
    # FUnktion hat need to be minimized
    #code
    return mu * scale_s_mc / scale_s_mcale_b_mc

m = Minuit(extended_likelihood, 
           mu=0.5, error_mu=0.1, limit_mu=(0.0, 3.0),
           errordef=0.5)
m.migrad()
best_mu = m.values["mu"]


q0 = np.zeros(len(mass_array))

#code

Die asymptotische Form der Wahrscheinlichkeitsdichte von $q_0$ wenn kein Signal erwartet wird lässt sich beschreiben durch:
$$ f(q_0|\hat{\mu}=0) \rightarrow \frac{1}{2}\left( \delta (q_0) + \chi^2_{1} \right) \, .$$
daraus lässt sich der $p_0$ Wert berechnen nach:
$$ p_0 = \int\limits_{q_{0_{\mathrm{obs}}}}^{\infty} \mathrm{d}q_0 f(q_0|\hat{\mu}=0) \, . $$
aus diesem kann die Signifikanz abschätzend berechnen zu: $$Z = \Phi^{-1}(1-p_0) = \sqrt{q_{0_{\mathrm{obs}}}} \, .$$ In diesem Fall sollte diese jedoch nur als eine erste Abschätzung betrachtet werden da die eingehende Parametrisierung des Signals ungenau ist.
Ebenso wird an dieser Stelle die Systematik nicht betrachtet, es fließt nur die statistische Unsicherheit in die Berechnung von $p_0$ bzw. $Z$ ein.

Im Jahr 2012 wurde der Teilchenüberschuss und die Zuordnung dieses Überschusses zu einem Higgs Boson das dann zu seiner Entdeckung führte mithilfe dieser Methode durchgeführt. 

Im Folgenden wird eine weitere Methode zur Bestimmung der Signifikanz vorgestellt, die mit anderen Annahmen zu ähnlichem Signifikanz wie in diesem Aufgabenteil führt und einen guten Crosscheck bzw. eine Alternative bietet.

Es ist sinnvoll das Zwischenergebnis abzuspeichern:

In [ ]:
q0, p0 = np.zeros(len(sigma_array)), np.zeros(len(sigma_array))
df = pd.DataFrame(data=np.array([mass_array, sigma_array, q0, p0, np.sqrt(q0)]).T,
                  columns=["mass_array", "gauss_sigma", "q0", "p0", "sqrt_q0"])

<div class="alert alert-info">

Visualisieren Sie die Ergebnisse. Für die Signifikanzniveaus kann die `get_p0_sigma_lines` aus `PlotHelper` verwendet werden.
</div>

In [ ]:
import matplotlib.pyplot as plt
from include.stattest.PlotHelper import PlotHelper

In [ ]:
fig, ax = plt.subplots(1,1)
PlotHelper.get_p0_sigma_lines(x_=mass_array, 
                             ax_obj_=ax, 
                             max_sigma_=4)
ax.plot(df.mass_array, df.p0+1)
plt.show()

# Betrachtung der finalen Verteilungen
------------------------------------------------------------------

Dieser Aufgabenteil ist eine Wiederholung und ist im ersten Aufgabenteil vorhanden - bietet aber eine gute Möglichkeit nochmal die Ausgangslage zu visualisieren - mit dem einzigen Unterschied das nun eine Begründung dafür existiert, warum genau die 125 GeV Signal MC Simulation verwendet wurde.


Es sind immer noch folgende Größen darstellbar (wobei es fraglich ist, ob `z1_index`, `z2_index`, sowie `z1_tag` bzw. `z2_tag` Größen sind die eine ausführlicher Betrachtung bedürfen):

In [ ]:
from include.processing.ApplyHelper import ProcessHelper
print(ProcessHelper.print_possible_variables("../data/for_long_analysis/mc_aftH/MC_2012_H_to_ZZ_to_4L_2el2mu_aftH.csv"))

<div class="alert alert-info">

Betrachten Sie die Verteilung der Vier-Leptonen-Invarianten Massen und Erklären sie den von der vier Leptonen Masse abhängigen $p_0$ - Wert im Zusammenhang mit den gemessenen Ereignissen. Warum gibt es keine Änderung des $p_0$ Wertes für größere Massen (Bei einer Wahl des Massenintervalls von $[105, 151]$)?
    
</div>

In [ ]:
from include.histogramm.HistOf import HistOf
import matplotlib.pyplot as plt

%matplotlib inline
plt.rcParams["figure.figsize"] = (12, 9)
h = HistOf(mc_dir="../data/for_long_analysis/mc_aftH",
           ru_dir="../data/for_long_analysis/ru_aftH", info=info)

In [ ]:
# Beispiel
h.variable("energy", 50, (0, 200))
ax = plt.gca()
ax.set_xlabel(r"$p_T$ in GeV")
ax.set_ylabel("Bineinträge")
plt.show()

Die hier dargestellten Unsicherheiten sind asymmetrisch. Hierzu wird von dem Erwartungswert eine Poisson Verteilung (gemessene Anzahl an Ereignissen) die untere Grenze ( - 34%) und die obere Grenze (+34%) bestimmt, womit ein 68% Unsicherheitsintervall angegeben werden kann. Die Asymmetrie der Poissonverteilung ist für einen kleinen Erwartungswert deutlich sichtbar. Erst im Grenzfall großer Erwartungswerte geht die Poissonverteilung in die Gaußverteilung über und die Unsicherheiten auf ein Messwert werden symmetrisch.

In [ ]:
# weitere Verteilungen

# Parametrisierung der Signal MC
----------------------

Nach der Wahl der passenden Signalhypothese soll die Verteilung genauer parametrisiert werden. Dies geschieht analog zu der Untergrundparametrisierung.

Die Begründung ist analog zu der in dem Abschnitt der Untergrundparametrisierung. Es änder sich lediglich die zur Parametrisierung verwendeten Funktionsformen: Diese sind in diesem Fall Glockenartig.

<div class="alert alert-info">

Parametrisieren Sie die Signalverteilung durch ein geeignetes Modell. Stellen Sie zudem sicher, dass das gewählte Modell das passendste ist.

</div>

Zur Auswahl stehen den Studenten:
- Gaußverteilung
- Cauchyverteilung
- Voigt-Profil
- Single - Sided - Crystal - Ball
- Double - Sided - Crystal - Ball

In [ ]:
from include.parameterization.SignalFunction import SignalFunction as SF

Variante:

1. <ins>Die Daten aus dem Histogramm werden roh ausgegeben.</ins>
<div class="alert alert-info">

    * Berechnen Sie die Unsicherheiten auf die einzelnen Bineinträge des skalierten Histogramms
    * Skalieren Sie die Histogramme der einzelnen Kanäle und fassen sie diese in einem gemeinsamen Histogramm zusammen.

    (Das Zugreifen auf die einzelnen Teile kann über den Attribut data geschehen, wobei das ein geschachteltes `dict` ist. Die erste Ebene enthält die drei einzelnen Kanäle. In jedem der Kanäle ist anschließend die unskalierten Histogrammeinträge, der Korrekturfaktor und die Daten, die für die Histogrammbildung verwendet wurden, gespeichert.)

</div>

In [ ]:
bins, hist_range, info = 999, (999.1, 999.2), [["2012"], ["B", "C"]]
mc_dir = "../data/for_long_analysis/mc_aftH/"

In [ ]:
raw_mc_data = HDG(bins=bins, hist_range=hist_range, 
           info=info, mc_dir=mc_dir).get_mc_raw(tag="signal")
print(list(raw_mc_data.data.keys()))
raw_mc_data.data["mc_track_H_ZZ_4el_2012"]

In [ ]:
import kafe2 as K2
# code

2. <ins>Die Daten aus dem Histogramm werden zusammengefasst ausgegeben</ins> 
    <div class="alert alert-info">
     
    Verwenden Sie die bereits zusammengefassten Größen für den Histogrammfit.

</div>

In [ ]:
format_data = McFitInit(bins=bins, hist_range=hist_range, tag="background", mc_dir=mc_dir)
bins, hist_range, hist_entries, combined_hist_entries_errors = format_data.variables_for_hist_fit

In [ ]:
import kafe2 as K2
# code

# Bestimmung der statistischen Signifikanz
----------------------

Nach der erfolgreichen Parametrisierung des Untergrundes und des Signals kann die Signifikanz des Signals bestimmt werden. Hierzu wird folgende Parametrisierung gewählt:
$$ f(x, \alpha_\mathrm{S}) = \alpha_\mathrm{S} f_{\mathrm{S}}(x|\theta_{\mathrm{S}}) + (1 - \alpha_\mathrm{S}) f_{\mathrm{U}}(x|\theta_{\mathrm{U}}) \, .$$

Der optimale Parameter $\alpha_{\mathrm{S}}$ entspricht der Signalstärke dieser Messung. Diese Bestimmung erfolgt mithilfe der eigenständig zu implementierenden $-2\ln\mathcal{L(\alpha_{\mathrm{S}})}$ - Funktion. Die zuvor bestimmten Parameter $\theta_{\mathrm{S}}$ und $\theta_{\mathrm{U}}$ der Wahrscheinlichkeitsdichte des Signals bzw. Untergrundes werden für die Bestimmung festgehalten (profiled Likelihood).

Die Betrachtung des Funktionswertes $-2\ln\mathcal{L}(\alpha_{\mathrm{S}} = 0)$ soll hierbei hervorgehoben werden. Dieser Entspricht nämlich der Annahme, dass in der vorhandenen Messung kein Signalanteil vorhanden ist und die Messung somit durch die Nullhypothese (nur Untergrund) erklärt werden kann. In asymptotischen Form lässt sich deshalb analog zu dem im Kapitel zur Parametrisierung der Signal MC Ausdruck die Signifikanz bestimmen mit: $$ Z = \sqrt{\frac{-2\ln\mathcal{L}(\alpha_{\mathrm{S}} = 0)}{\mathcal{L}_{\mathrm{min}}}} \, .$$

<div class="alert alert-info">
     
Implementieren Sie die $-2\ln\mathcal{L(\alpha_S, m_{4\ell})}$ Funktion (falls noch nicht vorhanden). Bestimmen Sie die Signalstärke zzgl. Unsicherheit der Messung. Geben Sie die Signifikanz an, mit der ein higgsartiges Boson gemessen wird. Geben Sie zudem die Masse des higgsartigen Bosons für die zweidimensionale Variante an und Vergleichen sie diese mit der Literatur. Visualisieren und diskutieren und vergleichen Sie Ihre Ergebnisse mit der ersten Methode.
    
Für die Minimierung kann wie bei QLT `iminuit` verwendet werden. Die numerische Integration kann wieder mithilfe von `scipy.integrate.quad` durchgeführt werden.
    
Ebenfalls ist es möglich einen `UnbinnedFit` mittels 'kafe2' durchzuführen und die extended Likelihood als Kostenfunktion selbstständig implementieren.
</div>

In [ ]:
# Messung:
from include.histogramm.HistDataGetter import HistDataGetter as HDG

bins, hist_range, info = 999, (999.1, 999.2), [["2012"], ["B", "C"]]
mc_dir = "../data/for_long_analysis/mc_aftH/" 
ru_dir = "../data/for_long_analysis/ru_aftH/"
raw_data = HDG(bins=bins, hist_range=hist_range, 
           info=info, mc_dir=mc_dir, ru_dir=ru_dir).get_data_raw("mass_4l")
raw_data.data

In [ ]:
# code iminuit

def extended_nll_iminuit(alpha_s, mass=125.0):
    # code
    return mass

In [ ]:
# code kafe2

def extended_nll_kafe2(data, model, parameter_values, parameter_constraints):
    """
    
    :param data: measurement array
    :param model: model array (calculate individual in # code)
    :param parameter_values:
    :param parameter_constraints: (not used)
    :return:
    """
    
    # check if one of the parameter_values is nan
    if any(f"{par}" == "nan" for par in parameter_values):
        return np.inf
    
    # your code
    
    _total_log_likelihood = 0.0 # your code
    
    # check if _total_log_likelihood is nan
    if np.isnan(_total_log_likelihood):
        return np.inf
    return -2.0 * _total_log_likelihood

## 1D profiled Likelihood
-----------------------------------

Die eindimensionale Variante variiert nur die Signalstärke. Die Masse wird nach der Parametrisierung festgehalten und nicht mehr variiert.

### Iminuit Variante

In [ ]:
# code (iminuit minimization)

Visualisierung:

In [ ]:
from include.stattest.PlotHelper import PlotHelper
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1.inset_locator import inset_axes, mark_inset

In [ ]:
alpha_ = np.linspace(0.0, 0.8, 1000)
nll_ = extended_nll(alpha_s=alpha_, mass=125)

# as example
nll_ = 150*(alpha_ - 0.4) ** 2

z_ = np.sqrt(nll_[0])

# code to further customize the plot

fig, ax = plt.subplots(1,1)
PlotHelper.plot_nll_scan_main_window(ax_obj_=ax, 
                                     x_=alpha_, 
                                     y_=nll_, 
                                     z_=z_)

ax_in = inset_axes(ax, width='50%', height='40%', loc="upper right")
PlotHelper.plot_nll_scan_inside_window(ax_obj_=ax_in, 
                                       x_=alpha_, y_=nll_, 
                                       x_ticks_=PlotHelper.get_error_points_from_nll(x_=alpha_, nll_=nll_))
mark_inset(ax, ax_in, loc1=2, loc2=4, fc="none", ec="grey", lw=0.5, alpha=0.5)
plt.show()

### kafe2 Variante

Für die `kafe2` Variante muss ein `UnbinnedContainer` erstellt und angepasst werden. Für die 1D Variante empfiehlt es sich die Methode `fix_parameter(par_name, value)` zu verwenden.
Die Signifikanz kann mithilfe der implementierten `extended_nll_kafe2` Kostenfunktion berechnet werden. Es ist empfehlenswert den Parameter `alpha` mittels `limit_parameter` auf einen sinnvollen Intervall zu beschränken.

In [ ]:
import kafe2 as K2

my_cost_function = K2.UnbinnedCostFunction_UserDefined(extended_nll_kafe2)

# more code

## 2D profiled Likelihood - Massenbestimmung
----------------------------------------------------------------

Abschließend wird auch die Masse bestimmt. Dies geschieht durch die zweidimensionale Variation – Masse und Signalstärke.

### Iminuit Variante

Ähnlich zu der 1D Variante kann auch gleichzeitig die beste Masse bestimmt werden. Das Vorgehen geschieht analog zu der 1D Variante/QLT Implementierung.

In [ ]:
# code:
#     - minimization
#     - plots

### kafe2 Variante

Der Unterschied  zu 1D Variante besteht nun lediglich darin, dass die Masse nicht mehr durch `fix_parameter` festgehalten wird, sondern in einem bestimmten Intervall (`limit_parameter`) best möglichst bestimmt wird.

In [ ]:
# code

## Kombination der Messungen: CMS und ATLAS
----------------------------------------------------------------

Um eine bessere Statistische Signifikanz zu erhalten ist es vom Vorteil weitere Messungen in die Gesamtbetrachtung hinzuzuziehen. Dies kann entweder durch die Hinzunahme weiterer Daten oder einem anderen Experiment erfolgen. Im Rahmen dieser Aufgabe wird das Zweite durchgeführt.

Für das ATLAS Experiment müssten deshalb ähnliche Schritte wie oben durchgeführt werden, wobei viele der Cuts jedoch Abweichungen aufweisen. Um die Arbeit etwas abzukürzen wird die finale Messung zur Verfügung gestellt, ebenso wie die Histogrammeinträge der MC Simulationen.

Zudem werden die Parameter der passendsten Funktionen (Linearkombination aus zentrierten Legendrepolynomen bis zur dritten Ordnung für den Untergrund und die DSCB Verteilung für das Signal) zur Verfügung gestellt (es ist jedoch ebenfalls möglich sich von diesen Tatsachen selbst zu überzeugen)

In [ ]:
import numpy as np
import pandas as pd
import kafe2 as K2
import os
from pprint import pprint

In [ ]:
atlas_data_dir_ = "../data/for_long_analysis/atlas_data_and_mc_hist/"
atlas_files = [os.path.join(atlas_data_dir_, file) for file in os.listdir(atlas_data_dir_) if ".csv" in file and "atlas" in file]
pprint(atlas_files)
hist_range_ = (106, 151)
num_bins = 45

data_A = np.loadtxt(atlas_files[2], delimiter=",")
data_A = data[(data >= hist_range_[0]) & (data <= hist_range_[1])]

mc_bac = np.loadtxt(atlas_files[1], delimiter=",").T
mc_bac_data = mc_bac[0]
mc_bac_error = mc_bac[1]

mc_sig = np.loadtxt(atlas_files[0], delimiter=",").T
mc_sig_data = mc_sig[0]
mc_sig_error = mc_sig[1]

In [ ]:
data_CMS = data
data_ATLAS = data_A

background_parameter_CMS = background_parameter
signal_parameter_CMS = signal_parameter

background_parameter_ATLAS  = np.array([0.025050083639971264, 
                                        0.00018744120591615378, 
                                        -1.178181555551107e-05, 
                                        1.1872474265844408e-07])
signal_parameter_ATLAS = np.array([2.1395802886491615, 
                                   124.19274632605674, 
                                   0.8324353801486924, 
                                   1.8034218165656613, 
                                   14.109501070824912, 
                                   19.9999999999894])
bfu_ATLAS = llc_background.grade_3
sfu_ATLAS = sf.DSCB

<div class="alert alert-info">
Kombinieren Sie die beiden Messungen und bestimmten sie die aus der Kombination resultierende Masse des Higgs Bosons, die die Signalwahrscheinlichkeit, sowie die Signifikanz, einer Messung des Higgs Bosons.

Sie können die von Ihnen bereits implementierte Funktionen weiterverwenden/modifizieren.
    
Hinweis:
    $$ \mathcal{L_{\mathrm{tot}}(\alpha_S, m_{4\ell})} = \mathcal{L_{1}(\alpha_S, m_{4\ell})} \mathcal{L_{2}(\alpha_S, m_{4\ell})} $$
</div>

In [ ]:
# code (1D and/or 2D minimization)
